# **Application of ML Algorithms Notebook - 2**

## **Data Preparation and Environment Setup**

Different Machiene Learning Classification Algorithms were applied on the dataset.

This is divided into 2 notebooks and this is the second one.

The classification Algorithms applied and compared in this notebook are 

1.   Ada Boost Classifier
2.   XGBoost Classifier
3.   Light GBM Classifier

As the data is highly skewed, all the above models are tried with and without resampling an the results are analysed.

Please Refer to Model-1.ipynb for detailed data preparation section

In [0]:
from google.colab import auth

In [0]:
!pip install gcloud

     |████████████████████████████████| 460kB 2.7MB/s 
  Created wheel for gcloud: filename=gcloud-0.18.3-cp36-none-any.whl size=602939 sha256=7937856f5d9ecd27ffabe075ae2e8a9bc76c6b51b465ef4ade7b607b14d280b0
  Stored in directory: /root/.cache/pip/wheels/b9/9b/9c/a01be401658fea33b93a35d03921b0c638266821b264dc8662
Successfully built gcloud


In [0]:
auth.authenticate_user()

In [0]:
!pip install gcsfs

In [0]:
d=[]
while(1):
  d.append('1')

In [0]:
!gcloud config set project skilful-orb-255314

Updated property [core/project].


In [0]:
!gsutil cp gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/test/three_class_model_train.csv /train.csv

Copying gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/test/three_class_model_train.csv...
\ [1 files][137.4 MiB/137.4 MiB]                                                
Operation completed over 1 objects/137.4 MiB.                                    


In [0]:
import pandas as pd
import numpy as np
import gcsfs

In [0]:
df_train = pd.read_csv('/train.csv')
df_train.head()

,region_en,category_name_en,parent_category_name_en,user_type,weekend,price,description_len,title_len,param_combined_len,image_present,image_top_1,deal_class_5
0,Sverdlovsk oblast,Children's products and toys,Personal belongings,Private,0,400.0,7,3,2,False,1008.0,Poor
1,Samara oblast,Furniture and interior,For the home and garden,Private,1,3000.0,7,3,1,False,692.0,Poor
2,Rostov oblast,Audio and video,Consumer electronics,Private,1,4000.0,17,2,5,False,3032.0,Okay
3,Tatarstan,Children's products and toys,Personal belongings,Company,0,2200.0,3,1,2,False,796.0,Good
4,Volgograd oblast,Cars,Transport,Private,0,40000.0,4,3,5,False,2264.0,Poor


In [0]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 12 columns):
region_en                  1503424 non-null object
category_name_en           1503424 non-null object
parent_category_name_en    1503424 non-null object
user_type                  1503424 non-null object
weekend                    1503424 non-null int64
price                      1503424 non-null float64
description_len            1503424 non-null int64
title_len                  1503424 non-null int64
param_combined_len         1503424 non-null int64
image_present              1503424 non-null bool
image_top_1                1503424 non-null float64
deal_class_5               1503424 non-null object
dtypes: bool(1), float64(2), int64(4), object(5)
memory usage: 127.6+ MB


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 
import time

In [0]:
X = df_train[['region_en','parent_category_name_en','user_type','weekend','price','description_len','title_len','param_combined_len','image_present','image_top_1']]
y = df_train[['deal_class_5']]

In [0]:
X_enc = pd.get_dummies(X, columns=['region_en','user_type','parent_category_name_en'], drop_first = True)
X_enc.head()

,weekend,price,description_len,title_len,param_combined_len,image_present,image_top_1,region_en_Bashkortostan,region_en_Belgorod oblast,region_en_Chelyabinsk oblast,region_en_Irkutsk oblast,region_en_Kaliningrad oblast,region_en_Kemerovo oblast,region_en_Khanty-Mansi Autonomous Okrug,region_en_Krasnodar Krai,region_en_Krasnoyarsk Krai,region_en_Nizhny Novgorod oblast,region_en_Novosibirsk oblast,region_en_Omsk oblast,region_en_Orenburg oblast,region_en_Perm Krai,region_en_Rostov oblast,region_en_Samara oblast,region_en_Saratov oblast,region_en_Stavropol Krai,region_en_Sverdlovsk oblast,region_en_Tatarstan,region_en_Tula oblast,region_en_Tyumen oblast,region_en_Udmurtia,region_en_Vladimir oblast,region_en_Volgograd oblast,region_en_Voronezh oblast,region_en_Yaroslavl oblast,user_type_Private,user_type_Shop,parent_category_name_en_Consumer electronics,parent_category_name_en_For business,parent_category_name_en_For the home and garden,parent_category_name_en_Hobbies & leisure,parent_category_name_en_Personal belongings,parent_category_name_en_Real estate,parent_category_name_en_Services,parent_category_name_en_Transport
0,0,400.0,7,3,2,False,1008.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,3000.0,7,3,1,False,692.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,1,4000.0,17,2,5,False,3032.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
3,0,2200.0,3,1,2,False,796.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,40000.0,4,3,5,False,2264.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size = 0.12, random_state = 0, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.12, random_state = 0, stratify=y_train)
sc = StandardScaler()
X_train.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.fit_transform(X_train[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_test.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_test[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_valid.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_valid[["price","description_len","title_len","param_combined_len","image_top_1"]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## **Modelling**

 **Sampling using SMOTE**

In [0]:
!pip install imblearn

In [0]:
  from imblearn.over_sampling import SMOTE
  import time

In [0]:
sm = SMOTE(sampling_strategy='auto',random_state=27)
y_train_smote = y_train.iloc[:, 0]
start = time.time()
X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train_smote)
end = time.time()
print("Time in seconds : ", end - start)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Time in seconds :  126.46320486068726


In [0]:
y_train_smote


array(['Poor', 'Poor', 'Poor', ..., 'Okay', 'Okay', 'Okay'], dtype=object)

Sample Weights:

In [0]:
from  sklearn.utils.class_weight import compute_class_weight
arr_y = np.unique(y.values.ravel())
sample_weights_data = compute_class_weight(class_weight = "balanced", classes = arr_y, y = y.values.ravel())

### Ada Boost

Ada Boost Without Resampling :

In [0]:
from sklearn.ensemble import AdaBoostClassifier

In [0]:
model = AdaBoostClassifier(n_estimators=100, random_state=42)
model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=42)

In [0]:
predictions = model.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.36      0.01      0.02     17217
        Okay       0.25      0.00      0.00      8730
        Poor       0.84      1.00      0.91    132815

    accuracy                           0.84    158762
   macro avg       0.48      0.34      0.31    158762
weighted avg       0.75      0.84      0.76    158762

[[   145      4  17068]
 [   102      3   8625]
 [   161      5 132649]]


Ada Boost With Resampling :

In [0]:
model = AdaBoostClassifier(n_estimators=100, random_state=42)
model.fit(X_train_smote,y_train_smote)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=42)

In [0]:
predictions = model.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.20      0.47      0.28     17217
        Okay       0.16      0.51      0.24      8730
        Poor       0.92      0.62      0.74    132815

    accuracy                           0.60    158762
   macro avg       0.43      0.53      0.42    158762
weighted avg       0.80      0.60      0.66    158762

[[ 8156  3897  5164]
 [ 2667  4453  1610]
 [31000 19393 82422]]


ADABoost With 100 stumps, Resampled data and Learning rate =  0.1

In [0]:
model = AdaBoostClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
model.fit(X_train_smote,y_train_smote)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.1,
                   n_estimators=100, random_state=42)

In [0]:
predictions = model.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.18      0.48      0.26     17217
        Okay       0.14      0.57      0.23      8730
        Poor       0.93      0.55      0.70    132815

    accuracy                           0.55    158762
   macro avg       0.42      0.53      0.40    158762
weighted avg       0.81      0.55      0.62    158762

[[ 8242  4872  4103]
 [ 2652  4993  1085]
 [34698 24595 73522]]


ADABoost didnot perform that well compared to other Boosting Algrithms

-----------------------------------------------------------

### XGBoost

In [0]:
import xgboost as xgb

XGBClassifier with 100 trees, Learning Rate 0.1, objective= 'multi:softprob', eval_metric = 'mlogloss' and without Resampling 

In [0]:
clf = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=10,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 verbosity = 3,
 objective= 'multi:softprob',
 eval_metric = 'mlogloss',
 sample_weight=sample_weights_data,
 seed=27)
start = time.time()
clf = clf.fit(X_train,y_train.values.ravel(),verbose=True)
end = time.time()
print("Time in seconds : ", end - start)

[17:56:05] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1078 extra nodes, 0 pruned nodes, max_depth=10
[17:56:09] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1270 extra nodes, 0 pruned nodes, max_depth=10
[17:56:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1396 extra nodes, 0 pruned nodes, max_depth=10
[17:56:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1186 extra nodes, 0 pruned nodes, max_depth=10
[17:56:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1066 extra nodes, 0 pruned nodes, max_depth=10
[17:56:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1142 extra nodes, 0 pruned nodes, max_depth=10
[17:56:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1272 extra nodes, 0 pruned nodes, max_depth=10
[17:56:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 

In [0]:
predictions = clf.predict(X_valid)
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

              precision    recall  f1-score   support

        Good       0.56      0.09      0.16     17217
        Okay       0.38      0.02      0.04      8730
        Poor       0.85      0.99      0.91    132815

    accuracy                           0.84    158762
   macro avg       0.60      0.37      0.37    158762
weighted avg       0.79      0.84      0.78    158762

[[  1567    126  15524]
 [   428    167   8135]
 [   795    144 131876]]


XgBoost using 60 Trees with Resampling an Learning Rate 0.1 with Smote Resampling

In [0]:
clf = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=60,
 max_depth=10,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 verbosity = 3,
 objective= 'multi:softprob',
 eval_metric = 'mlogloss',
 seed=27)
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote,verbose=True)
end = time.time()
print("Time in seconds : ", end - start)

[02:52:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1274 extra nodes, 0 pruned nodes, max_depth=10
[02:52:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1592 extra nodes, 0 pruned nodes, max_depth=10
[02:52:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 746 extra nodes, 0 pruned nodes, max_depth=10
[02:52:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1552 extra nodes, 0 pruned nodes, max_depth=10
[02:52:58] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1688 extra nodes, 0 pruned nodes, max_depth=10
[02:53:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1262 extra nodes, 0 pruned nodes, max_depth=10
[02:53:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 970 extra nodes, 0 pruned nodes, max_depth=10
[02:53:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 

In [0]:
predictions = clf.predict(X_valid.as_matrix())
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

        Good       0.24      0.43      0.31     17217
        Okay       0.17      0.57      0.26      8730
        Poor       0.93      0.69      0.79    132815

    accuracy                           0.66    158762
   macro avg       0.45      0.57      0.46    158762
weighted avg       0.81      0.66      0.71    158762

[[ 7482  4450  5285]
 [ 1935  4991  1804]
 [21370 19748 91697]]


XgBoost using 60 Trees with Resampling an Learning Rate 0.1 with Smote Resampling with max_depth = 15

In [0]:
clf = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=60,
 max_depth=15,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 verbosity = 3,
 objective= 'multi:softprob',
 eval_metric = 'mlogloss',
 seed=27)
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote,verbose=True)
end = time.time()
print("Time in seconds : ", end - start)

[03:42:36] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8908 extra nodes, 0 pruned nodes, max_depth=15
[03:42:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 15278 extra nodes, 0 pruned nodes, max_depth=15
[03:43:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3430 extra nodes, 0 pruned nodes, max_depth=15
[03:43:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 11390 extra nodes, 0 pruned nodes, max_depth=15
[03:44:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14834 extra nodes, 0 pruned nodes, max_depth=15
[03:44:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 9068 extra nodes, 0 pruned nodes, max_depth=15
[03:44:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6404 extra nodes, 0 pruned nodes, max_depth=15
[03:45:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning en

In [0]:
predictions = clf.predict(X_valid.as_matrix())
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

        Good       0.27      0.42      0.33     17217
        Okay       0.19      0.50      0.28      8730
        Poor       0.92      0.75      0.83    132815

    accuracy                           0.70    158762
   macro avg       0.46      0.56      0.48    158762
weighted avg       0.81      0.70      0.74    158762

[[  7308   3608   6301]
 [  2000   4387   2343]
 [ 17555  15091 100169]]


XgBoost using 100 Trees with Resampling an Learning Rate 0.1 with Smote Resampling with max_depth = 10

In [0]:
clf = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=10,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 verbosity = 3,
 objective= 'multi:softprob',
 eval_metric = 'mlogloss',
 sample_weight=sample_weights_data,
 seed=27)
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote,verbose=True)
end = time.time()
print("Time in seconds : ", end - start)

[18:21:16] ======== Monitor: Learner ========
[18:21:16] GetGradient: 1.16505s, 100 calls @ 11650us
[18:21:16] PredictRaw: 0.906931s, 100 calls @ 9069us
[18:21:16] UpdateOneIter: 1332.49s, 100 calls @ 13324884us
[18:21:16] ======== Monitor: GBTree ========
[18:21:16] BoostNewTrees: 1238.64s, 100 calls @ 12386424us
[18:21:16] CommitModel: 91.773s, 100 calls @ 917730us
[18:21:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1274 extra nodes, 0 pruned nodes, max_depth=10
[18:21:54] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1592 extra nodes, 0 pruned nodes, max_depth=10
[18:22:04] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 746 extra nodes, 0 pruned nodes, max_depth=10
[18:22:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1552 extra nodes, 0 pruned nodes, max_depth=10
[18:22:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1688 extra nodes, 

In [0]:
X_train_smote.size

128564436

In [0]:
predictions = clf.predict(X_valid.as_matrix())
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

        Good       0.25      0.42      0.32     17217
        Okay       0.18      0.53      0.27      8730
        Poor       0.92      0.72      0.81    132815

    accuracy                           0.68    158762
   macro avg       0.45      0.56      0.46    158762
weighted avg       0.81      0.68      0.73    158762

[[ 7242  4050  5925]
 [ 1947  4635  2148]
 [19534 17237 96044]]


**Best Model**

XgBoost using 100 Trees with Resampling an Learning Rate 0.1 with Smote Resampling with max_depth = 15

In [0]:
clf = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=15,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 verbosity = 3,
 objective= 'multi:softprob',
 eval_metric = 'mlogloss',
 seed=27)
start = time.time()
clf = clf.fit(X_train_smote,y_train_smote,verbose=True)
end = time.time()
print("Time in seconds : ", end - start)

[19:49:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8908 extra nodes, 0 pruned nodes, max_depth=15
[19:50:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 15278 extra nodes, 0 pruned nodes, max_depth=15
[19:50:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3430 extra nodes, 0 pruned nodes, max_depth=15
[19:51:09] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 11390 extra nodes, 0 pruned nodes, max_depth=15
[19:51:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14834 extra nodes, 0 pruned nodes, max_depth=15
[19:51:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 9068 extra nodes, 0 pruned nodes, max_depth=15
[19:52:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6404 extra nodes, 0 pruned nodes, max_depth=15
[19:52:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning en

In [0]:
predictions = clf.predict(X_valid.as_matrix())
print(classification_report(y_valid,predictions))
print(confusion_matrix(y_valid,predictions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

        Good       0.28      0.42      0.34     17217
        Okay       0.20      0.44      0.28      8730
        Poor       0.91      0.79      0.85    132815

    accuracy                           0.73    158762
   macro avg       0.47      0.55      0.49    158762
weighted avg       0.81      0.73      0.76    158762

[[  7193   2952   7072]
 [  2042   3801   2887]
 [ 16148  11803 104864]]


**Prediction on Test Set**

In [0]:
predictions = clf.predict(X_test.as_matrix())
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

        Good       0.28      0.42      0.34     19565
        Okay       0.20      0.43      0.27      9920
        Poor       0.91      0.79      0.85    150926

    accuracy                           0.73    180411
   macro avg       0.47      0.54      0.49    180411
weighted avg       0.81      0.73      0.76    180411

[[  8121   3570   7874]
 [  2339   4236   3345]
 [ 18203  13326 119397]]


-------------------------------------------------------------------------------

### Light GBM

In [0]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from time import time 

LightGBM doesnt need resampling as weights are handled well

In [0]:
cate_cols = ['region_en',  'parent_category_name_en', 'user_type']

In [0]:
for c in cate_cols:
    X[c] = LabelEncoder().fit_transform(X[c].values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
y = LabelEncoder().fit_transform(y.values)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.12, random_state = 0, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.12, random_state = 0, stratify=y_train)
sc = StandardScaler()
X_train.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.fit_transform(X_train[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_test.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_test[["price","description_len","title_len","param_combined_len","image_top_1"]])
X_valid.loc[:,["price","description_len","title_len","param_combined_len","image_top_1"]] = sc.transform(X_valid[["price","description_len","title_len","param_combined_len","image_top_1"]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Default LGBMClassifier with 100 trees

In [0]:
lgbm = LGBMClassifier(
          objective = "multiclass",
          num_class = 3,
          bagging_fraction = 0.9,  # subsample
          feature_fraction = 0.9,      # subsample_freq
          bagging_seed = 42,
          verbosity = 2,
          verbose_eval=10,
          n_estimators =100,
          class_weight = "balanced")

In [0]:
t = time()
print('Fitting Up')
lgbmodel = lgbm.fit(X_train,y_train,categorical_feature=cate_cols,eval_set = (X_valid, y_valid),verbose = 4,early_stopping_rounds=30)
print('Done')
print(f'Done, time = {time() - t}')

Fitting Up


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['parent_category_name_en', 'region_en', 'user_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds.
[4]	valid_0's multi_logloss: 1.00587
[8]	valid_0's multi_logloss: 0.953616
[12]	valid_0's multi_logloss: 0.920924
[16]	valid_0's multi_logloss: 0.901268
[20]	valid_0's multi_logloss: 0.888599
[24]	valid_0's multi_logloss: 0.880686
[28]	valid_0's multi_logloss: 0.874973
[32]	valid_0's multi_logloss: 0.871004
[36]	valid_0's multi_logloss: 0.868084
[40]	valid_0's multi_logloss: 0.865514
[44]	valid_0's multi_logloss: 0.863978
[48]	valid_0's multi_logloss: 0.862409
[52]	valid_0's multi_logloss: 0.861029
[56]	valid_0's multi_logloss: 0.860078
[60]	valid_0's multi_logloss: 0.858985
[64]	valid_0's multi_logloss: 0.858203
[68]	valid_0's multi_logloss: 0.857218
[72]	valid_0's multi_logloss: 0.856294
[76]	valid_0's multi_logloss: 0.855287
[80]	valid_0's multi_logloss: 0.854432
[84]	valid_0's multi_logloss: 0.853972
[88]	valid_0's multi_logloss: 0.853454
[92]	valid_0's multi_logloss: 0.852951
[96]	valid_0's multi_logloss: 0.852425
[100]

In [0]:
predictions = lgbm.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.23      0.50      0.31     19565
           1       0.16      0.62      0.26      9920
           2       0.95      0.63      0.75    150926

    accuracy                           0.61    180411
   macro avg       0.45      0.58      0.44    180411
weighted avg       0.83      0.61      0.68    180411

[[ 9768  5695  4102]
 [ 2480  6187  1253]
 [30514 25852 94560]]


LGBMClassifier with Learning Rate 0.01 and Max Depth = 15

In [0]:
lgbm = LGBMClassifier(
          objective = "multiclass",
          num_class = 3,
          num_leaves = 60,
          max_depth = 15,
          learning_rate = 0.01,
          bagging_fraction = 0.9,  # subsample
          feature_fraction = 0.9,  # colsample_bytree
          bagging_freq = 5,        # subsample_freq
          bagging_seed = 42,
          verbosity = 2,
          verbose_eval=10,
          n_estimators =100,
          class_weight = "balanced")

In [0]:
t = time()
print('Fitting Up')
lgbmodel = lgbm.fit(X_train,y_train,categorical_feature=cate_cols,eval_set = (X_valid, y_valid),verbose = 4,early_stopping_rounds=30)
print('Done')
print(f'Done, time = {time() - t}')

Fitting Up


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['parent_category_name_en', 'region_en', 'user_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds.
[4]	valid_0's multi_logloss: 1.08641
[8]	valid_0's multi_logloss: 1.07501
[12]	valid_0's multi_logloss: 1.06396
[16]	valid_0's multi_logloss: 1.05375
[20]	valid_0's multi_logloss: 1.04401
[24]	valid_0's multi_logloss: 1.03527
[28]	valid_0's multi_logloss: 1.02675
[32]	valid_0's multi_logloss: 1.01843
[36]	valid_0's multi_logloss: 1.01069
[40]	valid_0's multi_logloss: 1.0033
[44]	valid_0's multi_logloss: 0.99626
[48]	valid_0's multi_logloss: 0.989641
[52]	valid_0's multi_logloss: 0.983438
[56]	valid_0's multi_logloss: 0.977484
[60]	valid_0's multi_logloss: 0.971923
[64]	valid_0's multi_logloss: 0.966501
[68]	valid_0's multi_logloss: 0.961352
[72]	valid_0's multi_logloss: 0.956513
[76]	valid_0's multi_logloss: 0.951935
[80]	valid_0's multi_logloss: 0.947568
[84]	valid_0's multi_logloss: 0.943447
[88]	valid_0's multi_logloss: 0.939651
[92]	valid_0's multi_logloss: 0.935847
[96]	valid_0's multi_logloss: 0.932208
[100]	valid_0's 

In [0]:
predictions = lgbm.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.22      0.48      0.30     19565
           1       0.16      0.63      0.25      9920
           2       0.94      0.62      0.75    150926

    accuracy                           0.60    180411
   macro avg       0.44      0.57      0.43    180411
weighted avg       0.82      0.60      0.67    180411

[[ 9457  5880  4228]
 [ 2489  6209  1222]
 [31123 26960 92843]]


LGBMClassifier with Learning Rate 0.1 and max_depth = 15


In [0]:
lgbm = LGBMClassifier(
          objective = "multiclass",
          num_class = 3,
          num_leaves = 60,
          max_depth = 15,
          learning_rate = 0.1,
          bagging_fraction = 0.9,  # subsample
          feature_fraction = 0.9,  # colsample_bytree
          bagging_freq = 5,        # subsample_freq
          bagging_seed = 42,
          verbosity = 2,
          verbose_eval=10,
          n_estimators =100,
          class_weight = "balanced")

In [0]:
t = time()
print('Fitting Up')
lgbmodel = lgbm.fit(X_train,y_train,categorical_feature=cate_cols,eval_set = (X_valid, y_valid),verbose = 4,early_stopping_rounds=30)
print('Done')
print(f'Done, time = {time() - t}')

Fitting Up


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['parent_category_name_en', 'region_en', 'user_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds.
[4]	valid_0's multi_logloss: 1.00096
[8]	valid_0's multi_logloss: 0.94575
[12]	valid_0's multi_logloss: 0.911372
[16]	valid_0's multi_logloss: 0.890976
[20]	valid_0's multi_logloss: 0.877467
[24]	valid_0's multi_logloss: 0.86903
[28]	valid_0's multi_logloss: 0.862993
[32]	valid_0's multi_logloss: 0.859221
[36]	valid_0's multi_logloss: 0.856717
[40]	valid_0's multi_logloss: 0.854553
[44]	valid_0's multi_logloss: 0.85263
[48]	valid_0's multi_logloss: 0.851395
[52]	valid_0's multi_logloss: 0.850025
[56]	valid_0's multi_logloss: 0.848811
[60]	valid_0's multi_logloss: 0.847931
[64]	valid_0's multi_logloss: 0.847004
[68]	valid_0's multi_logloss: 0.846494
[72]	valid_0's multi_logloss: 0.845904
[76]	valid_0's multi_logloss: 0.845256
[80]	valid_0's multi_logloss: 0.844459
[84]	valid_0's multi_logloss: 0.843769
[88]	valid_0's multi_logloss: 0.843314
[92]	valid_0's multi_logloss: 0.842904
[96]	valid_0's multi_logloss: 0.842344
[100]	va

In [0]:
predictions = lgbm.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.24      0.51      0.32     19565
           1       0.17      0.62      0.26      9920
           2       0.95      0.64      0.76    150926

    accuracy                           0.62    180411
   macro avg       0.45      0.59      0.45    180411
weighted avg       0.83      0.62      0.69    180411

[[ 9966  5535  4064]
 [ 2503  6123  1294]
 [29773 25177 95976]]


LGBMClassifier with Learning Rate 0.2 and max_depth = 15

In [0]:
lgbm = LGBMClassifier(
          objective = "multiclass",
          num_class = 3,
          num_leaves = 60,
          max_depth = 15,
          learning_rate = 0.2,
          bagging_fraction = 0.9,  # subsample
          feature_fraction = 0.9,  # colsample_bytree
          bagging_freq = 5,        # subsample_freq
          bagging_seed = 42,
          verbosity = 2,
          verbose_eval=10,
          n_estimators =100,
          class_weight = "balanced")

In [0]:
t = time()
print('Fitting Up')
lgbmodel = lgbm.fit(X_train,y_train,categorical_feature=cate_cols,eval_set = (X_valid, y_valid),verbose = 4,early_stopping_rounds=30)
print('Done')
print(f'Done, time = {time() - t}')

Fitting Up


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['parent_category_name_en', 'region_en', 'user_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds.
[4]	valid_0's multi_logloss: 0.942759
[8]	valid_0's multi_logloss: 0.88968
[12]	valid_0's multi_logloss: 0.868342
[16]	valid_0's multi_logloss: 0.85906
[20]	valid_0's multi_logloss: 0.854054
[24]	valid_0's multi_logloss: 0.851708
[28]	valid_0's multi_logloss: 0.849196
[32]	valid_0's multi_logloss: 0.847231
[36]	valid_0's multi_logloss: 0.845934
[40]	valid_0's multi_logloss: 0.844297
[44]	valid_0's multi_logloss: 0.843331
[48]	valid_0's multi_logloss: 0.842646
[52]	valid_0's multi_logloss: 0.841411
[56]	valid_0's multi_logloss: 0.840201
[60]	valid_0's multi_logloss: 0.839599
[64]	valid_0's multi_logloss: 0.838795
[68]	valid_0's multi_logloss: 0.838181
[72]	valid_0's multi_logloss: 0.837647
[76]	valid_0's multi_logloss: 0.836662
[80]	valid_0's multi_logloss: 0.835541
[84]	valid_0's multi_logloss: 0.834938
[88]	valid_0's multi_logloss: 0.834251
[92]	valid_0's multi_logloss: 0.833914
[96]	valid_0's multi_logloss: 0.833366
[100]	

In [0]:
predictions = lgbm.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.24      0.52      0.33     19565
           1       0.17      0.61      0.26      9920
           2       0.95      0.64      0.77    150926

    accuracy                           0.63    180411
   macro avg       0.45      0.59      0.45    180411
weighted avg       0.83      0.63      0.69    180411

[[10077  5410  4078]
 [ 2537  6074  1309]
 [29377 24527 97022]]


Best Model

LGBMClassifier with Learning Rate 0.1 and max_depth = 15 and No of trees = 200

In [0]:
lgbm = LGBMClassifier(
          objective = "multiclass",
          num_class = 3,
          num_leaves = 60,
          max_depth = 15,
          learning_rate = 0.1,
          bagging_fraction = 0.9,  # subsample
          feature_fraction = 0.9,  # colsample_bytree
          bagging_freq = 5,        # subsample_freq
          bagging_seed = 42,
          verbosity = 2,
          verbose_eval=10,
          n_estimators =200,
          class_weight = "balanced")

In [0]:
t = time()
print('Fitting Up')
lgbmodel = lgbm.fit(X_train,y_train,categorical_feature=cate_cols,eval_set = (X_valid, y_valid),verbose = 4,early_stopping_rounds=30)
print('Done')
print(f'Done, time = {time() - t}')

Fitting Up


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['parent_category_name_en', 'region_en', 'user_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds.
[4]	valid_0's multi_logloss: 1.00096
[8]	valid_0's multi_logloss: 0.94575
[12]	valid_0's multi_logloss: 0.911372
[16]	valid_0's multi_logloss: 0.890976
[20]	valid_0's multi_logloss: 0.877467
[24]	valid_0's multi_logloss: 0.86903
[28]	valid_0's multi_logloss: 0.862993
[32]	valid_0's multi_logloss: 0.859221
[36]	valid_0's multi_logloss: 0.856717
[40]	valid_0's multi_logloss: 0.854553
[44]	valid_0's multi_logloss: 0.85263
[48]	valid_0's multi_logloss: 0.851395
[52]	valid_0's multi_logloss: 0.850025
[56]	valid_0's multi_logloss: 0.848811
[60]	valid_0's multi_logloss: 0.847931
[64]	valid_0's multi_logloss: 0.847004
[68]	valid_0's multi_logloss: 0.846494
[72]	valid_0's multi_logloss: 0.845904
[76]	valid_0's multi_logloss: 0.845256
[80]	valid_0's multi_logloss: 0.844459
[84]	valid_0's multi_logloss: 0.843769
[88]	valid_0's multi_logloss: 0.843314
[92]	valid_0's multi_logloss: 0.842904
[96]	valid_0's multi_logloss: 0.842344
[100]	va

**Prediction on Test data**

In [0]:
predictions = lgbm.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.24      0.51      0.33     19565
           1       0.17      0.62      0.26      9920
           2       0.95      0.64      0.77    150926

    accuracy                           0.63    180411
   macro avg       0.45      0.59      0.45    180411
weighted avg       0.83      0.63      0.69    180411

[[10055  5445  4065]
 [ 2512  6105  1303]
 [29185 24629 97112]]


### XgBoost with 100 trees Learning rate = 0.1 and max-depth = 15 did the best among all models